In [145]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [146]:
#Loading the Data
data=pd.read_csv('data/voice.csv')

In [175]:
#making the data
X_data=np.array(data.iloc[:,:20]).astype(np.float32)
Y_data=(np.array(data.iloc[:,20])=='male').astype(np.float32)

In [214]:
#Preprocessing of data
train_ind=np.random.choice(X_data.shape[0],2500,replace=False)
test_ind=np.delete(np.arange(0,3168),train_ind)
print(train_ind.shape,test_ind.shape)
X_train=X_data[train_ind].T
Y_train=Y_data[train_ind].reshape(1,-1)
X_test=X_data[test_ind].T
Y_test=Y_data[test_ind].reshape(1,-1)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(2500,) (668,)
(20, 2500) (1, 2500) (20, 668) (1, 668)


In [215]:
#Tensorflow graph
X=tf.placeholder(tf.float32,[20,None])
Y=tf.placeholder(tf.float32,[1,None])
W1=tf.Variable(tf.random_normal([200,20]),dtype=tf.float32)
b1=tf.Variable(tf.random_normal([200,1]),dtype=tf.float32)
W2=tf.Variable(tf.random_normal([1,200]),dtype=tf.float32)
b2=tf.Variable(tf.random_normal([1]),dtype=tf.float32)
hold_prob=tf.placeholder(tf.float32)
A1=tf.nn.dropout(tf.nn.relu(tf.matmul(W1,X)+b1),keep_prob=hold_prob)
Yhat=tf.matmul(W2,A1)+b2

In [216]:
#Define loss
loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y,logits=Yhat))
#acc=tf.reduce_sum(tf.cast(tf.equal(Y,tf.round(Yhat)),dtype=tf.float32))

In [217]:
#Optimizer
opt=tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [220]:
#Session
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10000+1):
        sess.run(opt,feed_dict={X:X_train,Y:Y_train,hold_prob:0.5})
        if i%2000==0:
            print(i,"Loss : ",sess.run(loss,feed_dict={Y:Y_test,X:X_test,hold_prob:1.0}),sess.run(loss,feed_dict={Y:Y_train,X:X_train,hold_prob:1.0}))

0 Loss :  163.06 225.569
2000 Loss :  0.266771 0.240439
4000 Loss :  0.156587 0.134313
6000 Loss :  0.145797 0.122571
8000 Loss :  0.141774 0.11708
10000 Loss :  0.135476 0.115044
